In [1]:
import logging
logging.captureWarnings(False)
logging.getLogger('py.warnings').setLevel(logging.ERROR)
from dask.distributed import Client, progress
client = Client(n_workers=5, threads_per_worker=4, memory_limit='100GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 20,Total memory: 465.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53178,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 465.66 GiB
Comm: tcp://127.0.0.1:53208,Total threads: 4
Dashboard: http://127.0.0.1:53209/status,Memory: 93.13 GiB
Nanny: tcp://127.0.0.1:53185,


In [2]:
import numpy as np
import torch
import subprocess
import sys
import torch_harmonics as th
import torch_harmonics.distributed as dist
import pandas as pd
import matplotlib.pyplot as plt
from subs1_utils import *

In [3]:
# For MacOS is higher than 12.3+
if torch.backends.mps.is_available():
    print("Running on GPU")
#    print("Running on CPU")
    device = torch.device("mps")
#    device = torch.device("cpu")
    print("MPS is activated:",torch.backends.mps.is_built()) # Was the current version of PyTorch built with MPS activated?
else:
    print("Running  on CPU")
    device = torch.device("cpu")

device

Running on GPU
MPS is activated: True


device(type='mps')

In [4]:
### This cell initializes the model
###
###     First Define all spectral grids
###
zw = 63 # zonal wave number
mw = 63 # meridional wave number
kmax = 26
imax = 192
jmax = 96
steps_per_day = 216*1.5 ### Changing this number impliws time step changes and should
#                       be implemented carefully
rescale = 16 # rescale interval for bred-vector approach to linear model
###
#
#
# provide experiment name and data path for writing out data
# datapath may need to be edited for your system
#
expname = 'TestNonLinearT63L26_Dist'
foo = str(subprocess.check_output(['whoami']))
end = len(foo) - 3
uname = foo[2:end]
datapath = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/'+expname+'/'
toffset = 0
#
# If restart see below settings
#
#toffset = 1800*1 # toffset is the number of days that have already run
#datapath_init = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/Restarts/Total/' # Location in restarts
datapath_init = datapath #set equal to datapath if restarting in same directory
#
if ( toffset == 0): # Cold Start
    subprocess.call(['rm','-r', datapath])
    subprocess.check_output(['mkdir', datapath])

rm: /Users/bkirtman/Work/AGCM/AGCM/tmp4/TestT63L26F_Dist/: No such file or directory


In [5]:
# Get the Gaussian latitudes and equally spaced longitudes
#
cost_lg, wlg, lats = precompute_latitudes(jmax)
lats = 90-180*lats/(np.pi)
lons = np.linspace(0.0,360.0-360.0/imax,imax)
#
# Instantiate grid to spectral (dsht) and spectral to grid (disht) distibuted transforms
#
vsht = th.RealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
dsht = dist.DistributedRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
disht = dist.DistributedInverseRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
dvsht = dist.DistributedRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
divsht = dist.DistributedInverseRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)

In [6]:
#
#
#
# Initialize spectral fields (at rest or to be read in)
def initialize(temp_newton,lnpsclim,kmax,mw,zw):
    for k in range (kmax):
        tmn1[k] = tmn1[k] + temp_newton[k]
        tmn2[k] = tmn2[k] + temp_newton[k]
        tmn3[k] = tmn3[k] + temp_newton[k]
    qmn1 = lnpsclim
    qmn2 = lnpsclim
    qmn3 = lnpsclim
    return tmn1,tmn2,tmn3,qmn1,qmn2,qmn3

In [7]:
#
# Initialization: Could read spectral restarts, or could
#              start at rest. If wanting to use grid point see
#              jupyter notebook preprocess
#              
#
# Implement at rest initial condition, but need coriolis since
# model predicts total vorticity
#
coriolis = np.broadcast_to([(4.0*np.pi/86400)*np.sin(lats[j,np.newaxis]*np.pi/180.0) for j in range(jmax)], (jmax, imax))
#
#
# Initialize spectral fields (at rest or to be read in)
# Need to read in background temperature data for Newtonian
# Relaxation and possible initialization, see preprocess
# for how to change source data or formulation.
#
temp_newton = torch.load('temp.spectral.pt') # newtonian 
#                                    relaxation temperature, spectral
lnpsclim = torch.load('lnps.spectral.pt') # lnps climatology for damping
#
# Read Climatology on Gausian Grid
#
tclim_gg = torch.load('tsig.ggrid.pt')
vortclim_gg = torch.load('vortsig.ggrid.pt')
divclim_gg = torch.load('divsig.ggrid.pt')
uclim_gg = torch.load('usig.ggrid.pt')
vclim_gg = torch.load('vsig.ggrid.pt')
dxqclim_gg = torch.load('dxq_gg.ggrid.pt')
dyqclim_gg = torch.load('dyq_gg.ggrid.pt')
#
divclim = dsht(divclim_gg)
vortclim = dsht(vortclim_gg)
tclim = dsht(tclim_gg)
#
zmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
zmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    zmn1 = torch.load(datapath_init+'zmn1.spectral.pt')
    zmn2 = torch.load(datapath_init+'zmn2.spectral.pt')
zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
dmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128) 
dmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    dmn1 = torch.load(datapath_init+'dmn1.spectral.pt')
    dmn2 = torch.load(datapath_init+'dmn2.spectral.pt')
dmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
tmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    tmn1 = torch.load(datapath_init+'tmn1.spectral.pt') 
    tmn2 = torch.load(datapath_init+'tmn2.spectral.pt')
tmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
wmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
qmn1 = torch.zeros((mw,zw),dtype=torch.complex128)
qmn2 = torch.zeros((mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    qmn1 = torch.load(datapath_init+'qmn1.spectral.pt')
    qmn2 = torch.load(datapath_init+'qmn2.spectral.pt')
qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
#
if ( toffset == 0 ): # Only do this if cold start
    tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 =\
    initialize(tclim,lnpsclim,kmax,mw,zw)
#
# Topography data - this should be spectral data or can be
#                        initialized to zero. If grid point data
#                        is desired see preprocess for how to
#                        convert to spectral. 
#
# Setting topography to zero here
#
phismn = torch.zeros((mw,zw),dtype=torch.complex128)
#
#
#
# Adding heating here see preprocess.ipynb
#
####heat = torch.load('heat.ggrid.pt')
#
#
# or set to zero
#
heat = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
#

In [8]:
###
### Constants, parameters, vertical differencing parameters,
### matricies for geopotential height, etc ...
###
delsig, si, sl, sikap, slkap, cth1, cth2, r1b, r2b = bscst(kmax)
### The above code is in subs1_utils.py - vertical structure related
### This code would need to be changed if the vertical resolution
### is changed - could be done by simply specifying delsig in bscst
###
amtrx, cmtrx, dmtrx = mcoeff(kmax,si,sl,slkap,r1b,r2b,delsig)
### The above code is for geopotential height and implicit scheme
### in subs1_utils.py but unlikely any changes would be needed
emtrx = inv_em(dmtrx,steps_per_day,kmax,mw,zw)
### The above code
### emtrix is used in the implicit time scheme, computed once here to save cpu time
### changes unlikely

In [ ]:
#### Preprocessing is complete - now time to run model
#
#
# The Model Runs in 30-day chuncks - need to specify how many 30-day chunks to run
tl = 30 ##### tl is the chunk size - typically 30 days, but for testing 3 is reasonable
#
# Suggested ichunk for time dependent models: 120
#
ae = 6.371E+06 # Earth radius
tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
zmnt = tmnt.detach().clone()
dmnt = tmnt.detach().clone()
qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
wmnt = tmnt.detach().clone()
#
ddtdiv = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
ddtvort = ddtdiv.detach().clone()
ttend = ddtdiv.detach().clone()
#
vort = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
div = ddtdiv .detach().clone()
temp = ddtdiv.detach().clone()
qdot = ddtdiv.detach().clone()
#
times = pd.date_range(start = '1950-01-01', end='2100-01-01', freq='D')
# 
ichunk = 60
#
idays = tl * ichunk
#
#
#
#
# Begin Time Loop
#
ii = 0
savedat = 0
daycount = 0
total_days = 0
nstep = idays*steps_per_day
while ii < nstep:
    ii = ii + 1
    savedat = savedat + 1
    zmnt[daycount] = zmnt[daycount] + zmn1/steps_per_day
    dmnt[daycount] = dmnt[daycount] + dmn1/steps_per_day
    tmnt[daycount] = tmnt[daycount] + tmn1/steps_per_day
    qmnt[daycount] = qmnt[daycount] + qmn1/steps_per_day
    wmnt[daycount] = wmnt[daycount] + wmn1/steps_per_day
    if (savedat == steps_per_day): # post processing
        #
        # Call Postprocessing Routine as needed
        #
        print((dmn1[10,2,2],dmn1[9,2,1]))
        daycount = daycount + 1
        total_days = total_days + 1
        print(['Day = ',total_days])
        if (daycount == tl):
            times_30day = times[total_days-tl+toffset:total_days+toffset]
            postprocessing(disht,divsht,zmnt,dmnt,tmnt,qmnt,wmnt,\
                           phismn,amtrx,times_30day,mw,zw,\
                           kmax,imax,jmax,sl,lats,lons,tl,datapath)
            tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            zmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            dmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
            wmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            daycount = 0
        savedat = 0
    #
    # Run model for one time step
    #
    # Spectral to grid transformation of needed fields:
    #   Vorticity, divergence, temperature, U, V, 
    #   grad(ln(Ps)), Q prescibed heating
    #
    #
    vort = disht(zmn2) ### This is the relative vorticity
    div = disht(dmn2)
    temp = disht(tmn2)
    qdot = disht(wmn2)
    u,v = uv(divsht,zmn2,dmn2,mw,zw,kmax,imax,jmax)
    dxq,dyq = gradq(divsht,qmn2,mw,zw,imax,jmax)
    #
    # Stack grid variables climo first[0] perturbation second[1]
    #
    vort2 = torch.stack((vortclim_gg,vort)) # vortclim is the absolute vorticity
    div2 = torch.stack((divclim_gg,div))
    temp2 = torch.stack((tclim_gg,temp))
    u2 = torch.stack((uclim_gg,u))
    v2 = torch.stack((vclim_gg,v))
    dxq2 = torch.stack((dxqclim_gg,dxq))
    dyq2 = torch.stack((dyqclim_gg,dyq))
    #
    # Non-Linear products
    #
    a,b,e,ut,vt,ri,wj,cbar,dbar = nlprod_prescribed_mean(u2,v2,vort2,div2,temp2,dxq2,dyq2,heat,coriolis,delsig,si,sikap,slkap,\
                                                                r1b,r2b,cth1,cth2,cost_lg,kmax,imax,jmax)
    #
    #
    # Grid to spectral transformation of nlprod results
    #
    ddtdiv,ddtvort = vortdivspec(vsht,a,b,kmax,mw,zw)
    zmn3 = - ddtvort
    dmn3 = ddtdiv - lap_sht(dsht,e,mw,zw)
    _,ttend = vortdivspec(vsht,ut,vt,kmax,mw,zw)
    #
    tmn3 = -ttend + dsht(ri)
    wmn3 = dsht(wj) ### Prescribed heating converted to spectral
    qmn3 = -dsht(cbar[1]) ### Only cbar here since dbar is included in implicit or explicit
    # 
    # Diffusion, Damping, Implicit or Explicit time differencing, Time filter
    #
    zmn3,dmn3,tmn3 = diffsn(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,mw,zw)
    #
    zmn3,dmn3,tmn3,qmn3 = damp_prescribed_mean(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,qmn1,qmn3,kmax,mw,zw)
    #
    dt = 86400.0/steps_per_day
    #
    zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 = \
                        explicit(dt,amtrx,cmtrx,dmtrx,emtrx,\
                        zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,\
                        tmn3,wmn1,wmn2,wmn3,qmn1,qmn2,qmn3,phismn,\
                        delsig,kmax,mw,zw)
    ####
    ## Reset zmn3, dmn3, tmn3,wmn3 & qmn3
    ###
    zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    dmn3 = zmn3.detach().clone()
    tmn3 = zmn3.detach().clone()
    wmn3 = zmn3.detach().clone()
    qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
# Done

(tensor(3.0017e-06-1.1438e-06j, dtype=torch.complex128), tensor(-5.5665e-07-5.9091e-07j, dtype=torch.complex128))
['Day = ', 1]
(tensor(-1.0418e-06+1.3389e-06j, dtype=torch.complex128), tensor(8.0601e-07-2.4468e-06j, dtype=torch.complex128))
['Day = ', 2]
(tensor(2.0642e-07-3.8372e-07j, dtype=torch.complex128), tensor(8.3800e-07-6.1440e-07j, dtype=torch.complex128))
['Day = ', 3]
(tensor(-2.1644e-06+4.0062e-07j, dtype=torch.complex128), tensor(-2.8387e-07+1.7677e-06j, dtype=torch.complex128))
['Day = ', 4]
(tensor(-8.1295e-09-1.9101e-07j, dtype=torch.complex128), tensor(-2.0131e-06-1.3786e-07j, dtype=torch.complex128))
['Day = ', 5]
(tensor(-1.7149e-06-1.8074e-06j, dtype=torch.complex128), tensor(-9.4216e-07-2.0482e-06j, dtype=torch.complex128))
['Day = ', 6]
(tensor(8.1856e-07-1.6663e-06j, dtype=torch.complex128), tensor(4.9352e-07-1.3261e-07j, dtype=torch.complex128))
['Day = ', 7]
(tensor(-9.4363e-08-8.4311e-07j, dtype=torch.complex128), tensor(1.3571e-07+4.5988e-07j, dtype=torch.co

(tensor(2.0918e-07-3.5264e-07j, dtype=torch.complex128), tensor(7.6323e-08-1.7958e-08j, dtype=torch.complex128))
['Day = ', 65]
(tensor(-2.5230e-08-5.6127e-07j, dtype=torch.complex128), tensor(5.4339e-08+4.6426e-08j, dtype=torch.complex128))
['Day = ', 66]
(tensor(-2.4525e-07-5.3532e-07j, dtype=torch.complex128), tensor(5.2682e-08+5.6202e-09j, dtype=torch.complex128))
['Day = ', 67]
(tensor(-2.5440e-07-4.5142e-07j, dtype=torch.complex128), tensor(5.0025e-08-4.1190e-08j, dtype=torch.complex128))
['Day = ', 68]
(tensor(-1.9816e-07-4.9338e-07j, dtype=torch.complex128), tensor(5.1946e-08-4.0042e-08j, dtype=torch.complex128))
['Day = ', 69]
(tensor(-2.4810e-07-5.7422e-07j, dtype=torch.complex128), tensor(4.8427e-08-3.2708e-08j, dtype=torch.complex128))
['Day = ', 70]
(tensor(-3.0678e-07-5.6403e-07j, dtype=torch.complex128), tensor(3.0779e-08-4.9365e-08j, dtype=torch.complex128))
['Day = ', 71]
(tensor(-2.7590e-07-4.5932e-07j, dtype=torch.complex128), tensor(3.8049e-08-2.5610e-08j, dtype=tor

(tensor(2.0919e-07-3.5253e-07j, dtype=torch.complex128), tensor(7.6302e-08-1.7987e-08j, dtype=torch.complex128))
['Day = ', 129]
(tensor(-2.5285e-08-5.6107e-07j, dtype=torch.complex128), tensor(5.4365e-08+4.6376e-08j, dtype=torch.complex128))
['Day = ', 130]
(tensor(-2.4538e-07-5.3518e-07j, dtype=torch.complex128), tensor(5.2776e-08+5.5869e-09j, dtype=torch.complex128))
['Day = ', 131]
(tensor(-2.5453e-07-4.5157e-07j, dtype=torch.complex128), tensor(5.0149e-08-4.1143e-08j, dtype=torch.complex128))
['Day = ', 132]
(tensor(-1.9813e-07-4.9400e-07j, dtype=torch.complex128), tensor(5.1985e-08-3.9870e-08j, dtype=torch.complex128))
['Day = ', 133]
(tensor(-2.4777e-07-5.7514e-07j, dtype=torch.complex128), tensor(4.8243e-08-3.2468e-08j, dtype=torch.complex128))
['Day = ', 134]
(tensor(-3.0624e-07-5.6453e-07j, dtype=torch.complex128), tensor(3.0356e-08-4.9267e-08j, dtype=torch.complex128))
['Day = ', 135]
(tensor(-2.7562e-07-4.5833e-07j, dtype=torch.complex128), tensor(3.7661e-08-2.5947e-08j, dt

(tensor(-2.6865e-08-7.5705e-07j, dtype=torch.complex128), tensor(-1.9767e-08+1.0899e-08j, dtype=torch.complex128))
['Day = ', 192]
(tensor(2.0920e-07-3.5254e-07j, dtype=torch.complex128), tensor(7.6297e-08-1.7983e-08j, dtype=torch.complex128))
['Day = ', 193]
(tensor(-2.5274e-08-5.6107e-07j, dtype=torch.complex128), tensor(5.4357e-08+4.6377e-08j, dtype=torch.complex128))
['Day = ', 194]
(tensor(-2.4538e-07-5.3516e-07j, dtype=torch.complex128), tensor(5.2769e-08+5.5810e-09j, dtype=torch.complex128))
['Day = ', 195]
(tensor(-2.5455e-07-4.5152e-07j, dtype=torch.complex128), tensor(5.0154e-08-4.1157e-08j, dtype=torch.complex128))
['Day = ', 196]
(tensor(-1.9817e-07-4.9394e-07j, dtype=torch.complex128), tensor(5.2010e-08-3.9885e-08j, dtype=torch.complex128))
['Day = ', 197]
(tensor(-2.4782e-07-5.7515e-07j, dtype=torch.complex128), tensor(4.8281e-08-3.2465e-08j, dtype=torch.complex128))
['Day = ', 198]
(tensor(-3.0624e-07-5.6466e-07j, dtype=torch.complex128), tensor(3.0376e-08-4.9229e-08j, d

(tensor(3.9458e-08-7.4427e-07j, dtype=torch.complex128), tensor(-7.7129e-09+1.6582e-08j, dtype=torch.complex128))
['Day = ', 255]
(tensor(-2.7517e-08-7.5620e-07j, dtype=torch.complex128), tensor(-1.9338e-08+1.0661e-08j, dtype=torch.complex128))
['Day = ', 256]
(tensor(2.0920e-07-3.5254e-07j, dtype=torch.complex128), tensor(7.6298e-08-1.7983e-08j, dtype=torch.complex128))
['Day = ', 257]
(tensor(-2.5274e-08-5.6108e-07j, dtype=torch.complex128), tensor(5.4357e-08+4.6378e-08j, dtype=torch.complex128))
['Day = ', 258]
(tensor(-2.4537e-07-5.3516e-07j, dtype=torch.complex128), tensor(5.2768e-08+5.5822e-09j, dtype=torch.complex128))
['Day = ', 259]
(tensor(-2.5454e-07-4.5152e-07j, dtype=torch.complex128), tensor(5.0152e-08-4.1157e-08j, dtype=torch.complex128))
['Day = ', 260]
(tensor(-1.9817e-07-4.9394e-07j, dtype=torch.complex128), tensor(5.2007e-08-3.9886e-08j, dtype=torch.complex128))
['Day = ', 261]
(tensor(-2.4782e-07-5.7514e-07j, dtype=torch.complex128), tensor(4.8281e-08-3.2469e-08j, d

...

(tensor(-1.9817e-07-4.9394e-07j, dtype=torch.complex128), tensor(5.2007e-08-3.9885e-08j, dtype=torch.complex128))
['Day = ', 885]
(tensor(-2.4782e-07-5.7514e-07j, dtype=torch.complex128), tensor(4.8280e-08-3.2468e-08j, dtype=torch.complex128))
['Day = ', 886]
(tensor(-3.0625e-07-5.6465e-07j, dtype=torch.complex128), tensor(3.0381e-08-4.9234e-08j, dtype=torch.complex128))
['Day = ', 887]
(tensor(-2.7554e-07-4.5857e-07j, dtype=torch.complex128), tensor(3.7627e-08-2.5881e-08j, dtype=torch.complex128))
['Day = ', 888]
(tensor(-1.2712e-07-4.5628e-07j, dtype=torch.complex128), tensor(3.9943e-08-7.7452e-09j, dtype=torch.complex128))
['Day = ', 889]
(tensor(-8.3090e-08-5.7634e-07j, dtype=torch.complex128), tensor(1.6821e-08-1.2422e-08j, dtype=torch.complex128))
['Day = ', 890]
(tensor(-1.1084e-07-6.0731e-07j, dtype=torch.complex128), tensor(-1.0727e-09+2.0268e-09j, dtype=torch.complex128))
['Day = ', 891]
(tensor(-5.3089e-08-5.4459e-07j, dtype=torch.complex128), tensor(-6.4088e-09-5.5155e-10j,

(tensor(-2.5454e-07-4.5152e-07j, dtype=torch.complex128), tensor(5.0152e-08-4.1157e-08j, dtype=torch.complex128))
['Day = ', 948]
(tensor(-1.9817e-07-4.9394e-07j, dtype=torch.complex128), tensor(5.2007e-08-3.9885e-08j, dtype=torch.complex128))
['Day = ', 949]
(tensor(-2.4782e-07-5.7514e-07j, dtype=torch.complex128), tensor(4.8280e-08-3.2468e-08j, dtype=torch.complex128))
['Day = ', 950]
(tensor(-3.0625e-07-5.6465e-07j, dtype=torch.complex128), tensor(3.0381e-08-4.9234e-08j, dtype=torch.complex128))
['Day = ', 951]
(tensor(-2.7554e-07-4.5857e-07j, dtype=torch.complex128), tensor(3.7627e-08-2.5881e-08j, dtype=torch.complex128))
['Day = ', 952]
(tensor(-1.2712e-07-4.5628e-07j, dtype=torch.complex128), tensor(3.9943e-08-7.7452e-09j, dtype=torch.complex128))
['Day = ', 953]
(tensor(-8.3090e-08-5.7634e-07j, dtype=torch.complex128), tensor(1.6821e-08-1.2422e-08j, dtype=torch.complex128))
['Day = ', 954]
(tensor(-1.1084e-07-6.0731e-07j, dtype=torch.complex128), tensor(-1.0727e-09+2.0268e-09j, 

(tensor(-2.4538e-07-5.3516e-07j, dtype=torch.complex128), tensor(5.2768e-08+5.5822e-09j, dtype=torch.complex128))
['Day = ', 1011]
(tensor(-2.5454e-07-4.5152e-07j, dtype=torch.complex128), tensor(5.0152e-08-4.1157e-08j, dtype=torch.complex128))
['Day = ', 1012]
(tensor(-1.9817e-07-4.9394e-07j, dtype=torch.complex128), tensor(5.2007e-08-3.9885e-08j, dtype=torch.complex128))
['Day = ', 1013]
(tensor(-2.4782e-07-5.7514e-07j, dtype=torch.complex128), tensor(4.8280e-08-3.2468e-08j, dtype=torch.complex128))
['Day = ', 1014]
(tensor(-3.0625e-07-5.6465e-07j, dtype=torch.complex128), tensor(3.0381e-08-4.9234e-08j, dtype=torch.complex128))
['Day = ', 1015]
(tensor(-2.7554e-07-4.5857e-07j, dtype=torch.complex128), tensor(3.7627e-08-2.5881e-08j, dtype=torch.complex128))
['Day = ', 1016]
(tensor(-1.2712e-07-4.5628e-07j, dtype=torch.complex128), tensor(3.9943e-08-7.7452e-09j, dtype=torch.complex128))
['Day = ', 1017]
(tensor(-8.3090e-08-5.7634e-07j, dtype=torch.complex128), tensor(1.6821e-08-1.2422e

(tensor(-2.5274e-08-5.6108e-07j, dtype=torch.complex128), tensor(5.4357e-08+4.6378e-08j, dtype=torch.complex128))
['Day = ', 1074]
(tensor(-2.4538e-07-5.3516e-07j, dtype=torch.complex128), tensor(5.2768e-08+5.5822e-09j, dtype=torch.complex128))
['Day = ', 1075]
(tensor(-2.5454e-07-4.5152e-07j, dtype=torch.complex128), tensor(5.0152e-08-4.1157e-08j, dtype=torch.complex128))
['Day = ', 1076]
(tensor(-1.9817e-07-4.9394e-07j, dtype=torch.complex128), tensor(5.2007e-08-3.9885e-08j, dtype=torch.complex128))
['Day = ', 1077]
(tensor(-2.4782e-07-5.7514e-07j, dtype=torch.complex128), tensor(4.8280e-08-3.2468e-08j, dtype=torch.complex128))
['Day = ', 1078]
(tensor(-3.0625e-07-5.6465e-07j, dtype=torch.complex128), tensor(3.0381e-08-4.9234e-08j, dtype=torch.complex128))
['Day = ', 1079]
(tensor(-2.7554e-07-4.5857e-07j, dtype=torch.complex128), tensor(3.7627e-08-2.5881e-08j, dtype=torch.complex128))
['Day = ', 1080]
(tensor(-1.2712e-07-4.5628e-07j, dtype=torch.complex128), tensor(3.9943e-08-7.7452e

(tensor(2.0920e-07-3.5254e-07j, dtype=torch.complex128), tensor(7.6298e-08-1.7983e-08j, dtype=torch.complex128))
['Day = ', 1137]
(tensor(-2.5274e-08-5.6108e-07j, dtype=torch.complex128), tensor(5.4357e-08+4.6378e-08j, dtype=torch.complex128))
['Day = ', 1138]
(tensor(-2.4538e-07-5.3516e-07j, dtype=torch.complex128), tensor(5.2768e-08+5.5822e-09j, dtype=torch.complex128))
['Day = ', 1139]
(tensor(-2.5454e-07-4.5152e-07j, dtype=torch.complex128), tensor(5.0152e-08-4.1157e-08j, dtype=torch.complex128))
['Day = ', 1140]
(tensor(-1.9817e-07-4.9394e-07j, dtype=torch.complex128), tensor(5.2007e-08-3.9885e-08j, dtype=torch.complex128))
['Day = ', 1141]
(tensor(-2.4782e-07-5.7514e-07j, dtype=torch.complex128), tensor(4.8280e-08-3.2468e-08j, dtype=torch.complex128))
['Day = ', 1142]
(tensor(-3.0625e-07-5.6465e-07j, dtype=torch.complex128), tensor(3.0381e-08-4.9234e-08j, dtype=torch.complex128))
['Day = ', 1143]
(tensor(-2.7554e-07-4.5857e-07j, dtype=torch.complex128), tensor(3.7627e-08-2.5881e-

In [ ]:
## Write spectral data for possible restart
##
torch.save(zmn1,datapath+'zmn1.spectral.pt')
torch.save(zmn2,datapath+'zmn2.spectral.pt')
torch.save(dmn1,datapath+'dmn1.spectral.pt')
torch.save(dmn2,datapath+'dmn2.spectral.pt')
torch.save(tmn1,datapath+'tmn1.spectral.pt')
torch.save(tmn2,datapath+'tmn2.spectral.pt')
torch.save(qmn1,datapath+'qmn1.spectral.pt')
torch.save(qmn2,datapath+'qmn2.spectral.pt')